In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Here is the code for training. If you want to do training by yourself, you should do some settings as follows:


1.   all filepathes:

> *   project filepath
*   styleimage filepath
*   training data filepath

2.   alpha and beta value for the loss function

> *   loss_toltal = alpha*loss_style+beta*loss_content
*   defaut: alpha = 9.0, beta=1.0
*   Remenber that to make the generated image close to style image is naturaly hader than to make it close to it's original image. Set the alpha higer than beta is reasonable.
*   Higer ratio of alpha/beta means you need more training data.


In [ ]:
import sys
sys.path.append('/content/gdrive/My Drive/Styletransfer')

from torch.utils.data import DataLoader
import torchvision.datasets as dset
import torch
import torch.nn
import torch.optim as optim
import torchvision.transforms as transforms
import transform_network
import lossModule
import dataload
from PIL import Image
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

projectdir = "/content/gdrive/My Drive/Styletransfer/"
#dataset
path2data="/content/gdrive/My Drive/Styletransfer/data/coco2014/train2014part"
transform_iamges = transforms.Compose([transforms.Resize((256,256)),
                                       transforms.ToTensor()])

#style data
styleimagedir = "/content/gdrive/My Drive/Styletransfer/style/"
styleimagename = 'la_muse'
styleimage = Image.open(styleimagedir+styleimagename+".jpg")
transform_style = transforms.Compose([transforms.Resize((256,256)),
                                transforms.ToTensor(),
                                ])

styleimage = transform_style(styleimage).to(device)
styleimage = torch.unsqueeze(styleimage, dim=0)

vgg16 = lossModule.Vgg16()
vgg16.eval()
vgg16.to(device)

alpha = 9.0
beta = 1.0
transformnet = transform_network.TransformNet().to(device)
lossnet = lossModule.Loss_Net(alpha,beta).to(device)

#checkpoints
stralpha = str(alpha).replace('.','_')
strbeta = str(beta).replace('.','_')
resultdir = projectdir+styleimagename+stralpha+strbeta+'result'
if not os.path.exists(resultdir):
  os.mkdir(resultdir)

checkpointspath = resultdir+'/checkpoints/'
if not os.path.exists(checkpointspath):
  os.mkdir(checkpointspath)

print('training begin')
transformnet.train(True)
optimizer = optim.Adam(transformnet.parameters(),lr = 0.001)
checkpoints = os.listdir(checkpointspath)


for epoch in range(2):
  for k in range(8):
    checkpointpath = 'dataset'+str(k)+'epoch'+str(epoch)+'checpoint.pth'
    if len(checkpoints)>0 and checkpointpath in checkpoints:
        transformnet.load_state_dict(torch.load(checkpointspath+checkpointpath))
        pass
    else:
        datasetpath = path2data+str(k)+'/'
        datset = dataload.CreateNiiDataset(datasetpath,transform=transform_iamges)
        coco_loader =DataLoader(datset,batch_size=4, shuffle=True,num_workers=4,pin_memory=True)
        sumlossvalue = 0.0
        for (i,data) in enumerate(coco_loader,0):
            inputs = data
            inputs = inputs.to(device)
            optimizer.zero_grad()
            outputs = transformnet(inputs)
            outputs = (outputs+1)/2.0
            #TODO:get ys y=output, yc=input# not tested yet
            ysyyc = torch.cat((styleimage,outputs,inputs),dim=0)
            loss = lossnet(ysyyc,vgg16)
            sumlossvalue +=loss
            if(i%1000==999):
              print('epoch:',epoch,'dataset:',k,',batchnumber:',i,',sumlossvalue:',sumlossvalue)
              sumlossvalue=0.0
            loss.backward()
            optimizer.step()
        PATH = checkpointspath+checkpointpath
        torch.save(transformnet.state_dict(), PATH)
'''
#network test: one time forward pass
cocoiter = iter(coco_loader)
images = cocoiter.next()
inputs= images.to(device)
print(inputs.shape)
output = transformnet(inputs)
print(styleimage.shape)
print(output.shape)
print(inputs.shape)
ysyyc = torch.cat((styleimage,output,inputs),dim=0)
print(ysyyc.shape)
loss = lossnet(ysyyc,vgg16)
loss.backward()
print(loss)
'''





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Here is the code for evaluation. The generated images will be saved in the folder of checheckpoints. You need to set the testimage path and the path of the checkpoints' folder.

In [2]:
#evaluation
get_ipython().__class__.__name__ = "ZMQInteractiveShell"

import sys
sys.path.append('/content/gdrive/My Drive/Styletransfer')

from torch.utils.data import DataLoader
import torchvision.datasets as dset
import torch
import torchvision.transforms as transforms
import transform_network
import lossModule
import dataload
from PIL import Image
import os


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

testimagename = 'IMG_2142'#'IMG_2138'#'IMG_0510'#'IMG_2144'#'IMG_2142''IMG_2138'#
testimagedir = '/content/gdrive/My Drive/Styletransfer/testimages/'
testimagepath = testimagedir+testimagename+'.JPG'
transform_iamges = transforms.Compose([transforms.Resize((256,256)),
                                       transforms.ToTensor()])
testimage = transform_iamges(Image.open(testimagepath)).to(device)
testimage = styleimage = torch.unsqueeze(testimage, dim=0)
n,c,h,w = testimage.shape
print(n,c,h,w)
n,c,h,w = testimage.shape
if(c==1):
  testimage = testimage.repeat(1,3,1,1)
n,c,h,w = testimage.shape
print(n,c,h,w)

checkpointspath = '/content/gdrive/My Drive/Styletransfer/starry_night_crop9_50_5result/checkpoints/'
resultpath = checkpointspath+'testresult/'
if not os.path.exists(resultpath):
  os.mkdir(resultpath)

transfernet = transform_network.TransformNet().to(device)
checkpoints = os.listdir(checkpointspath)

trans2image = transforms.ToPILImage()

for epoch in range(2):
  for k in range(8):
    checkpointpath = 'dataset'+str(k)+'epoch'+str(epoch)+'checpoint.pth'
    if checkpointpath in checkpoints:
      transfernet.load_state_dict(torch.load(checkpointspath+checkpointpath))#, map_location=torch.device('cpu')
      transfernet.eval()
      transferedimage = transfernet(testimage)
      transferedimage = (transferedimage+1)/2.0
      transferedimage=transferedimage.to('cpu')
      transferedimage = trans2image(transferedimage[0,:,:,:])
      transferedimage.save(resultpath +'result'+testimagename+str(epoch)+str(k)+'.jpg')
      print(transferedimage.size)

cuda:0
1 3 256 256
1 3 256 256
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)


In [ ]:
from PIL import Image 
im=Image.open('/content/gdrive/My Drive/Styletransfer/style/composition_vii.jpg')
im